import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# ^^^ pyforest auto-imports - don't write above this line
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

$$
\renewcommand{\like}{{\cal L}}
\renewcommand{\loglike}{{\ell}}
\renewcommand{\err}{{\cal E}}
\renewcommand{\dat}{{\cal D}}
\renewcommand{\hyp}{{\cal H}}
\renewcommand{\Ex}[2]{E_{#1}[#2]}
\renewcommand{\x}{{\mathbf x}}
\renewcommand{\v}[1]{{\mathbf #1}}
$$

# Clinical Deterioration Prediction Model - Selection of Ensemble Algorithms 

## Data

The final dataset used for the inferential statistics project includes unique ICU admission of 46,234 patients’ demographic (age), vital (blood pressure, heart rate, body temperature, and Glasgow Comma Scale), underlying conditions (HIV, metastatic cancer, and hematologic malignancy), admission type (scheduled surgical, medical, or unscheduled surgical), renal (urinary output, and Blood Urea Nitrogen), and others (serum bicarbonate level, sodium level, potassium level, and bilirubin level) data. This dataset is build based on the commonly used mortality prediction tool, Simplified Acute Physiology Score II (SAPSII). 

In [1]:
os.getcwd()

<IPython.core.display.Javascript object>

'C:\\Users\\abebu\\Dropbox\\Data Science\\Projects\\Capstone Project 1\\Potential Projects\\9. MIMIC\\Machine Learning\\Clinical-Deterioration-Prediction-Model---Bayesian-Linear-Regression'

In [3]:
os.chdir("C://Users/abebu/Google Drive/mimic-iii-clinical-database-1.4")
saps = pd.read_csv("saps_ts.csv", header=0, index_col=0)
saps.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,los,hdeath,death,admission,ud,bun,Bicarbonate,...,WBC_3.0,hr_0.0,hr_2.0,hr_4.0,hr_7.0,hr_11.0,bp_0.0,bp_2.0,bp_5.0,bp_13.0
0,268,110404,280836,3.2490,1,1,8,0.0,6.0,0.0,...,0,0,0,0,0,1,0,0,0,1
1,269,106296,206613,3.2788,0,0,8,17.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
2,270,188028,220345,2.8939,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
3,271,173727,249196,2.0600,0,0,8,0.0,0.0,0.0,...,0,1,0,0,0,0,1,0,0,0
4,272,164716,210407,1.6202,0,0,8,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0


In [4]:
saps_e=saps.drop(['los','death','SUBJECT_ID','HADM_ID', 'ICUSTAY_ID'], axis=1)
saps_e

,hdeath,admission,ud,bun,Bicarbonate,ventilation,Temp,Bilirubin,gcs,AGE,...,WBC_3.0,hr_0.0,hr_2.0,hr_4.0,hr_7.0,hr_11.0,bp_0.0,bp_2.0,bp_5.0,bp_13.0
0,1,8,0.0,6.0,0.0,6.0,0.0,0.0,26.0,12.0,...,0,0,0,0,0,1,0,0,0,1
1,0,8,17.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,18.0,...,0,0,0,0,0,1,0,0,0,1
3,0,8,0.0,0.0,0.0,6.0,3.0,0.0,0.0,7.0,...,0,1,0,0,0,0,1,0,0,0
4,0,8,0.0,0.0,0.0,0.0,3.0,0.0,0.0,12.0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61112,0,8,0.0,0.0,0.0,6.0,3.0,0.0,26.0,16.0,...,0,0,1,0,0,0,0,0,0,1
61113,0,8,0.0,0.0,0.0,0.0,3.0,0.0,5.0,18.0,...,0,0,1,0,0,0,1,0,0,0
61114,0,0,0.0,0.0,0.0,6.0,3.0,0.0,7.0,7.0,...,0,1,0,0,0,0,0,0,1,0
61115,0,0,0.0,0.0,0.0,6.0,3.0,0.0,26.0,12.0,...,0,1,0,0,0,0,0,0,0,1


In [12]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
np.random.seed(42)

 
# Matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['figure.figsize'] = (9, 9)

import seaborn as sns

# Scipy helper functions
from scipy.stats import percentileofscore
from scipy import stats

In [10]:
# Standard ensembel ML Models for comparison

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Splitting data into training/testing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

# Distributions
import scipy

In [14]:

# PyMC3 for Bayesian Inference
import pymc3 as pm

In [5]:

# grade and returns training and testing datasets
def format_data(df):
    # Target is hospital death
    labels = saps_e['hdeath']
    
    # Drop target (hdeath) from features
    df = df.drop(columns=['hdeath'])
    
    # Split into training/testing sets with 30% split
    X_train, X_test, y_train, y_test = train_test_split(df, labels, 
                                                        test_size = 0.30,
                                                        random_state=42)
    
    return X_train, X_test, y_train, y_test


In [6]:
X_train, X_test, y_train, y_test = format_data(saps_e)
X_train.head()


<IPython.core.display.Javascript object>

,admission,ud,bun,Bicarbonate,ventilation,Temp,Bilirubin,gcs,AGE,UO,...,WBC_3.0,hr_0.0,hr_2.0,hr_4.0,hr_7.0,hr_11.0,bp_0.0,bp_2.0,bp_5.0,bp_13.0
53545,8,0.0,0.0,0.0,0.0,3.0,0.0,0.0,15.0,0.0,...,0,1,0,0,0,0,1,0,0,0
51512,8,0.0,6.0,0.0,0.0,3.0,0.0,0.0,18.0,0.0,...,0,0,1,0,0,0,0,0,1,0
23837,8,0.0,0.0,0.0,0.0,3.0,0.0,0.0,12.0,0.0,...,0,0,1,0,0,0,0,0,1,0
21929,0,0.0,10.0,0.0,6.0,3.0,0.0,5.0,18.0,0.0,...,0,1,0,0,0,0,0,0,0,1
57339,8,0.0,0.0,0.0,0.0,3.0,0.0,0.0,12.0,0.0,...,0,0,1,0,0,0,0,0,1,0


In [7]:
print(X_train.shape)
print(X_test.shape)

(42781, 27)
(18336, 27)


### Standard Ensemble Machine Learning Models

In [13]:
# Evaluate several ml models by training on training set and testing on testing set
def evaluate(X_train, X_test, y_train, y_test):
    #Names of models
    model_name_list = ['Random Forest', 'Extra Trees',
                       'Gradient Boosted']
    model = np.arange(1, 4)
    train_accuracy = np.empty(len(model))
    test_accuracy = np.empty(len(model))
    # Instantiate the models
    model1 = RandomForestClassifier(n_estimators=100)
    model2 = ExtraTreesClassifier(n_estimators=100)
    model3 = GradientBoostingClassifier(n_estimators=100)
    results =pd.DataFrame(columns=['train_accuracy', 'test_accuracy'])    
    
    # Train and predict with each model
    for i, model in enumerate([model1, model2, model3]):
        model.fit(X_train, y_train)
        
        train_accuracy=model.score(X_train, y_train)
        test_accuracy=model.score(X_test, y_test)
        
        model_name=model_name_list[i]
        results.loc[model_name, :] =[train_accuracy, test_accuracy]
    return results
    

In [14]:
results = evaluate(X_train, X_test, y_train, y_test)
results

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,train_accuracy,test_accuracy
Random Forest,0.949323,0.91983
Extra Trees,0.949323,0.919339
Gradient Boosted,0.924149,0.925993
